In [ ]:
cd drive/MyDrive/네카라쿠배/딥러닝/프로젝트

[Errno 2] No such file or directory: 'drive/MyDrive/네카라쿠배/딥러닝/프로젝트'
/content/drive/MyDrive/네카라쿠배/딥러닝/프로젝트


In [ ]:
ls

baseline.ipynb       checkpoiont_128.pt  open_224/  resnet50_v2.pt
baseline_result.csv  open/               open_448/  resnet50_ver1_sj.ipynb


# 데이콘 제공 data 로드 Baseline

## 사용 package 선언

In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from torch import optim
from torch import nn

from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision import transforms

import random
from glob import glob
import pandas as pd
import numpy as np
from PIL import Image

## 데이터 관련 함수 정의 및 데이터 셋 선언

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import transformers

from glob import glob
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision import transforms


def extract_day(file_name):
    day = int(file_name.split('.')[-2][-2:])
    return day


def make_day_array(image_pathes):
    day_array = np.array([extract_day(file_name) for file_name in image_pathes])
    return day_array


def make_image_path_array(root_path=None):
    if root_path is None:
        bc_directories = glob('./BC/*')
        lt_directories = glob('./LT/*')

    else:
        bc_directories = glob(root_path + 'BC/*')
        lt_directories = glob(root_path + 'LT/*')

    bc_image_path = []
    for bc_path in bc_directories:
        images = glob(bc_path + '/*.png')
        bc_image_path.extend(images)

    lt_image_path = []
    for lt_path in lt_directories:
        images = glob(lt_path + '/*.png')
        lt_image_path.extend(images)

    return bc_image_path, lt_image_path


def make_dataframe(root_path=None):
    bc_image_path, lt_image_path = make_image_path_array(root_path)
    bc_day_array = make_day_array(bc_image_path)
    lt_day_array = make_day_array(lt_image_path)

    bc_df = pd.DataFrame({'file_name': bc_image_path,
                          'day': bc_day_array})
    bc_df['species'] = 'bc'

    lt_df = pd.DataFrame({'file_name': lt_image_path,
                          'day': lt_day_array})
    lt_df['species'] = 'lt'

    total_data_frame = pd.concat([bc_df, lt_df]).reset_index(drop=True)

    return total_data_frame


def make_combination(length, species, data_frame):
    before_file_path = []
    after_file_path = []
    time_delta = []

    for i in range(length):
        sample = data_frame[data_frame['species'] == species].sample(2)
        after = sample[sample['day'] == max(sample['day'])].reset_index(drop=True)
        before = sample[sample['day'] == min(sample['day'])].reset_index(drop=True)

        before_file_path.append(before.iloc[0]['file_name'])
        after_file_path.append(after.iloc[0]['file_name'])
        delta = int(after.iloc[0]['day'] - before.iloc[0]['day'])
        time_delta.append(delta)

    combination_df = pd.DataFrame({
        'before_file_path': before_file_path,
        'after_file_path': after_file_path,
        'time_delta': time_delta,
    })

    combination_df['species'] = species

    return combination_df


class KistDataset(Dataset):
    def __init__(self, combination_df, is_test= None):
        self.combination_df = combination_df
        self.transform = transforms.Compose([
            transforms.Resize(224),
            transforms.ToTensor()
        ])
        self.is_test = is_test

    def __getitem__(self, idx):
        before_image = Image.open(self.combination_df.iloc[idx]['before_file_path']) #before_file_path
        after_image = Image.open(self.combination_df.iloc[idx]['after_file_path'])  #after_file_path

        before_image = self.transform(before_image)
        after_image = self.transform(after_image)
        if self.is_test:
            return before_image, after_image
        time_delta = self.combination_df.iloc[idx]['time_delta']
        return before_image, after_image, time_delta

    def __len__(self):
        return len(self.combination_df)

## Data load

In [ ]:
def seed_everything(seed): # seed 고정
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)


seed_everything(2048)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
lr = 1e-5
epochs = 10
batch_size = 64
valid_batch_size = 50

# model = CompareNet().to(device)    # 모델 선언
# optimizer = optim.Adam(model.parameters(), lr=lr)  # 옵티마이저 선언

########################################################################################################################
## 이미지 증강 부분
from torchvision.transforms.transforms import RandomRotation
# 간단한 데이터 augmentation

data_transforms = {
    'train_dataset' : transforms.Compose([transforms.Resize([256,256]),
                                  transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(),
                                  transforms.RandomCrop(224), transforms.RandomRotation(45), transforms.ToTensor(),
                                  transforms.Normalize([0.458,0.456,0.406],[0.229,0.224,0.225]) ]),
#     'val' : transforms.Compose([transforms.Resize([256,256]),
#                                 transforms.RandomCrop(224), transforms.RandomRotation(45), transforms.ToTensor(),
#                                 transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])  ])
}

import os
from torchvision.datasets import ImageFolder

data_dir = './open_224'

image_datasets = {x : ImageFolder(root = os.path.join(data_dir, x),
                                  transform = data_transforms[x]) for x in ['train_dataset']}

dataloaders = {x : torch.utils.data.DataLoader(image_datasets[x],
                                               batch_size = batch_size,
                                               shuffle = True,
                                               num_workers = 2) for x in ['train_dataset']}

dataset_sizes = {x : len(image_datasets[x]) for x in ['train_dataset']}
########################################################################################################################

total_dataframe = make_dataframe(root_path = "./open_224/train_dataset/")

bt_combination = make_combination(5000, 'bc', total_dataframe)
lt_combination = make_combination(5000, 'lt', total_dataframe)

bt_train = bt_combination.iloc[:4500]
bt_valid = bt_combination.iloc[4500:]

lt_train = lt_combination.iloc[:4500]
lt_valid = lt_combination.iloc[4500:]

train_set = pd.concat([bt_train, lt_train])
valid_set = pd.concat([bt_valid, lt_valid])



train_dataset = KistDataset(train_set)
valid_dataset = KistDataset(valid_set)

train_data_loader = DataLoader(train_dataset,
                               batch_size=batch_size,
                               shuffle=True)

valid_data_loader = DataLoader(valid_dataset,
                               batch_size=valid_batch_size)


In [ ]:
train_set

,before_file_path,after_file_path,time_delta,species
0,./open_224/train_dataset/BC/BC_02/DAT26.png,./open_224/train_dataset/BC/BC_06/DAT37.png,11,bc
1,./open_224/train_dataset/BC/BC_01/DAT11.png,./open_224/train_dataset/BC/BC_01/DAT23.png,12,bc
2,./open_224/train_dataset/BC/BC_02/DAT24.png,./open_224/train_dataset/BC/BC_01/DAT33.png,9,bc
3,./open_224/train_dataset/BC/BC_09/DAT07.png,./open_224/train_dataset/BC/BC_09/DAT29.png,22,bc
4,./open_224/train_dataset/BC/BC_02/DAT04.png,./open_224/train_dataset/BC/BC_07/DAT31.png,27,bc
...,...,...,...,...
4495,./open_224/train_dataset/LT/LT_07/DAT11.png,./open_224/train_dataset/LT/LT_03/DAT34.png,23,lt
4496,./open_224/train_dataset/LT/LT_06/DAT28.png,./open_224/train_dataset/LT/LT_08/DAT29.png,1,lt
4497,./open_224/train_dataset/LT/LT_08/DAT05.png,./open_224/train_dataset/LT/LT_10/DAT08.png,3,lt
4498,./open_224/train_dataset/LT/LT_01/DAT13.png,./open_224/train_dataset/LT/LT_07/DAT24.png,11,lt


In [ ]:
bt_combination.head()

,before_file_path,after_file_path,time_delta,species
0,./open_224/train_dataset/BC/BC_02/DAT26.png,./open_224/train_dataset/BC/BC_06/DAT37.png,11,bc
1,./open_224/train_dataset/BC/BC_01/DAT11.png,./open_224/train_dataset/BC/BC_01/DAT23.png,12,bc
2,./open_224/train_dataset/BC/BC_02/DAT24.png,./open_224/train_dataset/BC/BC_01/DAT33.png,9,bc
3,./open_224/train_dataset/BC/BC_09/DAT07.png,./open_224/train_dataset/BC/BC_09/DAT29.png,22,bc
4,./open_224/train_dataset/BC/BC_02/DAT04.png,./open_224/train_dataset/BC/BC_07/DAT31.png,27,bc


In [ ]:
total_dataframe.head()

,file_name,day,species
0,./open_224/train_dataset/BC/BC_01/DAT04.png,4,bc
1,./open_224/train_dataset/BC/BC_01/DAT09.png,9,bc
2,./open_224/train_dataset/BC/BC_01/DAT03.png,3,bc
3,./open_224/train_dataset/BC/BC_01/DAT10.png,10,bc
4,./open_224/train_dataset/BC/BC_01/DAT12.png,12,bc


In [ ]:
for step, (before_image, after_image, time_delta) in tqdm(enumerate(train_data_loader)):
    print(step)
    print(before_image)
    print(after_image)
    print(time_delta)
    break

## 모델 선언


In [ ]:
class Resnet(nn.Module):

    def __init__(self, embedding_dimension=1, pretrained=True):
        super().__init__()
        self.model = models.resnet50(pretrained = pretrained)
        
        input_features_fc_layer = self.model.fc.in_features
        self.model.fc = nn.Linear(input_features_fc_layer, embedding_dimension, bias = False)
        self.before_net = self.model
        self.after_net = self.model
        
    def forward(self, before_input, after_input):
        before = self.before_net(before_input)
        after = self.after_net(after_input)
        delta = before - after
        return delta
    

In [ ]:
class Resnet(nn.Module):

    def __init__(self):
        super(Resnet, self).__init__()
        self.before_net = models.resnet50(pretrained=True)
        self.after_net = models.resnet50(pretrained=True)
        
    def forward(self, before_input, after_input):
        before = self.before_net(before_input)
        after = self.after_net(after_input)
        delta = before - after
        return delta
    

### Pretrained model load (resnet50)

In [ ]:
## pretrained 된 모델들을 불러올 수 있다
import torch
from torch import nn
from torchvision import models

resnet = Resnet()  # True = weights도 같이
# num_ftrs = resnet.fc.in_features        # 출력층의 최종 클래스 갯수
# resnet.fc = nn.Linear(num_ftrs, 1)     # pretrained 된 최종 클래스 수와 우리 데이터의 최종 클래스 수를 맞춰주기
resnet = resnet.to(device)

criterion = nn.L1Loss()
optimizer_ft = optim.Adam(filter(lambda p : p.requires_grad, resnet.parameters()), lr = 0.001)
# 마지막으로 최종 클래스 수를 변환하는 과정에서 학습하기 위한 코드

from torch.optim import lr_scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma = 0.1)

In [ ]:
i = 0
for child in resnet.children():
    print(i)
    print(child)
    print('-'*30)
    i += 1

0
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1

In [ ]:
## 특정 레이어는 고정시키기
## 입력에 가까운 레이어 순으로

ct = 0

for child in resnet.children():
    ct += 1

    if ct < 6:  # 0~5번까지는 그대로, 6~9번 레이어는 학습을 해라
        for param in child.parameters():
            param.requires_grad = True  # False가 맞는거 같은데 오류 발생함 (위에서 살펴봤을때, layer 개수가 10개가 아니고 1개로 나옴)

In [ ]:
for epoch in tqdm(range(epochs)):
    for step, (before_image, after_image, time_delta) in tqdm(enumerate(train_data_loader)):
        before_image = before_image.to(device)
        after_image = after_image.to(device)
        time_delta = time_delta.to(device)

        optimizer_ft.zero_grad()
        logit = resnet(before_image, after_image)
        print(logit.squeeze(1).float())
        print(logit.float().shape)
        
        print(time_delta.float())
        print(time_delta.float().shape)

        train_loss = (torch.sum(torch.abs(logit.squeeze(1).float() - time_delta.float())) /
                      torch.LongTensor([batch_size]).squeeze(0).to(device))
        print(type(train_loss))
        train_loss.backward()
        optimizer_ft.step()

        if step % 15 == 0:
            print('\n=====================loss=======================')
            print(f'\n=====================EPOCH: {epoch}=======================')
            print(f'\n=====================step: {step}=======================')
            print('MAE_loss : ', train_loss.detach().cpu().numpy())

    valid_losses = []
    with torch.no_grad():
        for valid_before, valid_after, time_delta in tqdm(valid_data_loader):
            valid_before = valid_before.to(device)
            valid_after = valid_after.to(device)
            valid_time_delta = time_delta.to(device)


            logit = resnet(valid_before, valid_after)
            valid_loss = (torch.sum(torch.abs(logit.squeeze(1).float() - valid_time_delta.float())) /
                          torch.LongTensor([valid_batch_size]).squeeze(0).to(device))
            valid_losses.append(valid_loss.detach().cpu())


    print(f'VALIDATION_LOSS MAE : {sum(valid_losses)/len(valid_losses)}')
    checkpoiont = {
        'model': resnet.state_dict(),

    }



In [ ]:
torch.save(checkpoiont, 'resnet50_ver1.pt')

## Predict test data set

In [ ]:
transform_resNet = transforms.Compose([
    transforms.Resize([256,256]),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

test_resNet = ImageFolder(root = './data/splitted/test', transform = transform_resNet)
test_loader_resNet = torch.utils.data.DataLoader(test_resNet,
                                                 batch_size = batch_size,
                                                 shuffle = True,
                                                 num_workers = 2)


In [ ]:
resnet50 = torch.load('./code/resnet50.pt')
resnet50.eval()

test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('ResNet test acc : ', test_accuracy)

ResNet test acc :  96.8212415856395
